# ДЗ. Gensim и tfidf

### Таня Казакова

# Gensim

In [1]:
!pip install nltk

In [2]:
!pip install spacy

In [3]:
!pip  install pyLDAvis

In [4]:
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tbkazakova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [44]:
import math
import collections

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
# print(df.target_names.unique())
# df.head()

In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(len(data_words))

11314


In [10]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)

In [11]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# print(data_lemmatized[:1])

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [15]:
# Не работает. Разрешили не делать.
mallet_path = 'mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

CalledProcessError: Command 'mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/91/b5c9lg_s0c9bslz7x5586n040000gp/T/15ba09_corpus.txt --output /var/folders/91/b5c9lg_s0c9bslz7x5586n040000gp/T/15ba09_corpus.mallet' returned non-zero exit status 1.

In [16]:
# Функция, считающая лучшее количество топиков
def get_best_num_topics(corpus, id2word):
    best_num_topics = 0
    best_coherence_lda = 0
    for i in range(3,80,9):
        print(i)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=i, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)
        if coherence_lda > best_coherence_lda:
            best_coherence_lda = coherence_lda
            best_num_topics = i
    return best_num_topics


In [ ]:
# Можно запустить функцию.
# Но она очень долго грузится. Код работает.
best_num_topics = get_best_num_topics(corpus, id2word)
print('Лучшее количество', best_num_topics)

Решим, что best_num_topics = 9. (Coherence Score = 0,529) (Вручную было попробовано много вариантов.)

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=9, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.529042485995545


In [18]:
# Получаем более удобное собрание "главных" слов для топиков
topics = lda_model.show_topics(formatted=False, num_words= 10)
doc_lda = lda_model[corpus]
# print(topics)

In [19]:
# Узнаём главный топик для каждого текста
textnumber_btopic = []
textnumber = 0
for text_list_lem in data_lemmatized:
    maxtopves = 0
    btopic = ' '
    for topic in topics:
        ntopic = topic[0]
        topves = 0
        for tupl in topic[1]:
            for lem in text_list_lem:
                if lem == tupl[0]:
                    topves += tupl[1]
            
        if topves > maxtopves:
            maxtopves = topves
            btopic = ntopic

    textnumber_btopic.append((textnumber, btopic))
    textnumber += 1
# print(textnumber_btopic)

In [20]:
# Получаем список номеров текстов для каждого топика
btopic_textnumber = []
for i in range(0, len(topics)):
    list_textnumbers = []
    for tupl in textnumber_btopic:
        if tupl[1] == i:
            list_textnumbers.append(tupl[0])
    btopic_textnumber.append((i, list_textnumbers))
# print(btopic_textnumber[0])

In [21]:
# Получаем список текстов (списков лемм) для каждого топика
btopic_texts = []
for btopic in btopic_textnumber:
    list_texts = []
    for textnumber in btopic[1]:
        list_texts.append(data_lemmatized[textnumber])
    btopic_texts.append((btopic[0], list_texts))
# print(len(btopic_texts))
# print(len(btopic_texts[2][1]))

# TF_IDF

In [22]:
import pandas as pd

In [23]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tbkazakova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
# Получаем словарь uniqueWords для каждого топика
btopic_uniqueWords = {}
for btopic in btopic_texts:
    uniqueWords = set()
    for list_words in btopic[1]:
        uniqueWords = set(uniqueWords).union(set(list_words))
    btopic_uniqueWords[btopic[0]] = uniqueWords

In [25]:
# Даём номер топика, список словарей слово-колво_вхождений
def get_listdicts_for_topic(ntop):
    uniqueWords = btopic_uniqueWords[ntop]
    listdicts = []
    for list_words in btopic_texts[ntop][1]:
        numOfWords_text = dict.fromkeys(uniqueWords, 0)
        for word in list_words:
            numOfWords_text[word] += 1
        listdicts.append(numOfWords_text)
    return listdicts

In [26]:
# Тут kernel начал умирать.

In [27]:
def computeTF(numOfWords, list_lem):
    tfDict = {}
    bagOfWordsCount = len(list_lem)
    for word, count in numOfWords.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [35]:
def computeIDF(documents):
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [45]:
def computeTFIDF(tf_text, idfs):
    tfidf = collections.Counter()
    for word, val in tf_text.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [88]:
# Получаем список списков с лемматизированным текстом, номером топика, 5 словами с самым большим tfidf
# для каждого текста в одном широком топике
def get_tfidftexts(top):
    i_text = 0
    tfidf_texts = []
    idfs = computeIDF(get_listdicts_for_topic(top))
    for list_lem in btopic_texts[top][1]:
        numOfWords = get_listdicts_for_topic(top)[i_text]
        tf_text = computeTF(numOfWords, list_lem)
        tfidf_text = computeTFIDF(tf_text, idfs)
        tfidf_5_text = tfidf_text.most_common(5)
        words5 = []
        for tup in tfidf_5_text:
            words5.append(tup[0])
        
        tfidf_texts.append((list_lem, top, words5))
        
        i_text += 1
    return tfidf_texts

In [89]:
# Записываем всё в файл
for top in range(0, len(topics)):
    print(top)
    info_texts = get_tfidftexts(top)
    with open("hw3_tdidf.csv", "a", encoding="utf-8") as f:
        for info_text in info_texts:
            string = ' '.join(info_text[0]) +', '+ str(info_text[1]) +', '+ ' '.join(info_text[2]) +'\n'
            f.write(string)
### Я подождала 2 часа. Аня разрешила не ждать, когда все топики пройдут, остановить работу ячейки.
### В папке файл с информацией по текстам первых трёх топиков

0
1
2
3


KeyboardInterrupt: 

### Про coherence score

Оценка согласованности смотрит, насколько хорошо разделились тексты по топикам, насколько тексты одного топика близки.
Чем больше согласованность, тем лучше.

Получаем (например, благодаря gensim) топики.

Разделяем тексты по топикам.

Считаем вероятности чего-нибудь (слов, биграмм, триграмм и т.д.) в текстах.

Чем ближе ближе вероятности, тем ближе тексты.